In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import matplotlib.pyplot as plt

# Fashion MNIST 데이터셋 로드
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.fashion_mnist.load_data()

# 클래스 이름
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 1us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
# 흑백(28x28) → RGB(96x96x3)으로 변환하는 함수 정의
def preprocess_images(images):
    images = tf.expand_dims(images, -1)  # (28,28) → (28,28,1)
    images = tf.image.resize(images, [96, 96])  # 이미지 크기 조정
    images = tf.image.grayscale_to_rgb(images)  # RGB 채널로 변환
    return images / 127.5 - 1

train_images_rgb = preprocess_images(train_images)
test_images_rgb = preprocess_images(test_images)

print(train_images_rgb.shape)  # (60000, 96, 96, 3)

In [ ]:
base_model = tf.keras.applications.MobileNetV2(input_shape=(96,96,3),
                                               include_top=False,
                                               weights='imagenet')




In [ ]:
# 베이스 모델의 상위 일부 레이어 학습 가능으로 설정
base_model.trainable = True

# 상위 레이어만 미세조정 가능하도록 설정
fine_tune_at = 100  # 이 지점 이후의 레이어만 학습
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

# Transfer Learning을 위한 모델 구성
model = tf.keras.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(10, activation='softmax')
])

model.summary()

In [ ]:
# 매우 작은 학습률로 재컴파일
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# 미세조정 학습 수행
fine_tune_history = model.fit(train_images_rgb, train_labels,
                              epochs=5,
                              validation_data=(test_images_rgb, test_labels))


In [ ]:
loss, accuracy = model.evaluate(test_images_rgb, test_labels)
print(f'Fine-tuned Test Accuracy: {accuracy:.4f}')

In [ ]:
# 정확도 변화 그래프
plt.figure(figsize=(8, 4))
plt.plot(history.history['accuracy'], label='Train accuracy')
plt.plot(history.history['val_accuracy'], label='Validation accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Transfer Learning Accuracy')
plt.legend()
plt.show()
